<a href="https://colab.research.google.com/github/safreitas2000/ml-analytics-mvp/blob/main/machine-learning/ML_MVP_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##MVP | Disciplina Machine Learning & Analytics - Modelo de Machine Learning - Letra a.
Aluno: Sérgio Augusto Freitas Filho</br></br>


###1. Definição do Problema

**Qual é a descrição do problema?**
A tarefa consiste em predizer em qual das 2 faixas de renda determinado indivíduou pertence, trata-se de um problema de classificação com 2 classes:  ">50k" ou "<=50K". A hipótese a ser testada é referente à participação minoritária de pessoas da raça negra e mulheres no grupo de mais alta renda.

**Dataset:** Salary Prediction Classification (https://www.kaggle.com/datasets/ayessa/salary-prediction-classification) <br/>
O dataset escolhido contém informações sobre características individuais, incluindo cor da pele, sexo, escolaridade, nacionalidade, ocupação, entre outras.


In [1]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_selection import SelectKBest # para a Seleção Univariada
from sklearn.feature_selection import f_classif # para o teste ANOVA da Seleção Univariada
from sklearn.feature_selection import RFE # para a Eliminação Recursiva de Atributos
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

####Visualização do DataSet

In [2]:
# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/safreitas2000/ml-analytics-mvp/main/salary.csv"

# Lê o arquivo
dataset = pd.read_csv(url, delimiter=',')

dataset.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


### 2. Preparação dos Dados

1. Realizei um pré-processamento manual do DataSet, onde exclui linhas com dados faltantes e marcados com "?";
2. Inicialmente implementei a classe LabelEncoder para codificar os atributos object (texto) em valores numéricos, entretanto, verifiquei que a atribuição dos números com o LabelEncoder pode criar uma ordem artificial que poderia influenciar indevidamente alguns algoritmos, para mitigar esse risco optei pela codificação **OneHotEncoder**;
3. Realizada a separação do dataset em treino e teste. Utilizado padrão de 80% da base para treino e 20% para teste.
4. Utilizada estratégia de validação cruzada com 10 partições, tornando o processo de treinamento mais robusto e contribuindo para capacidade de generalização dos modelos .
5. Utilizado SelectKBest para seleção dos melhores atributos para predição, foram realizados testes comparativos com diferentes valores para k, optamos por 66 com score acima de 10k, valores manores para K apresentou perda de confiança.



In [3]:
# Aplicação do OneHotEncoder no Dataset
categorical_columns = dataset.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse=False) # Cria uma instância do OneHotEncoder
encoded_columns = encoder.fit_transform(dataset[categorical_columns])
encoded_column_names = encoder.get_feature_names_out(categorical_columns)
dataset_encoded = pd.DataFrame(encoded_columns, columns=encoded_column_names) # Cria um novo DataFrame com as colunas codificadas
dataset_encoded = pd.concat([dataset.drop(columns=categorical_columns), dataset_encoded], axis=1)# Concatena as colunas codificadas com o DataFrame original
dataset_encoded = dataset_encoded.drop(columns=['salary_ >50K'])#eliminei a coluna duplicada do target que não é necessária.

# Visualização preliminar das colunas escolhidas para teste da hipótese
selected_columns = dataset_encoded[["race_ Black", "sex_ Female", "salary_ <=50K"]]

# Exibindo as primeiras linhas das colunas selecionadas
selected_columns.head(10)

,race_ Black,sex_ Female,salary_ <=50K
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,0.0,0.0,1.0
3,1.0,0.0,1.0
4,1.0,1.0,1.0
5,0.0,1.0,1.0
6,1.0,1.0,1.0
7,0.0,0.0,0.0
8,0.0,1.0,0.0
9,0.0,0.0,0.0


O dataset tinha originalmente 14 atributos, com o OneHotEncoded foi para 108. Dos 108 atributos selecionei os 10 melhores _scores com o SelectKBest. Ainda na função de Feature Selection, peguei os 10 melhores _scores e extrapolei para pegar todas as features criadas pelo HotEncoded referente ao atributo original. Essa última ação é fundamental para manter a coerência do dataset considerando todas as categorias para aqueles considerados mais relevantes.
Dessa forma decidimos por 37 atributos para treinamento dos modelos.

No Feature Selection selecionei os 10 atributos _scores mais altos com SelectKBest:
['age' 'education-num' 'capital-gain' 'hours-per-week' 'marital-status_ Married-civ-spouse' 'marital-status_ Never-married'
'relationship_ Husband' 'relationship_ Own-child' 'sex_ Female' 'sex_ Male']
Executei um filtro no DataSet (hotencoded) para considerar todos os atributos com as palavras abaixo, de forma a não excluir qualquer categoria da feature original.
palavras_chave = ['age', 'education', 'capital', 'hours', 'marital', 'relationship', 'sex']


> Bloco com recuo



In [15]:
#Primeiro vamos realizar a seleção de atributos com a base original e antes de aplicar o OneHotEncoder.
seed = 7

array = dataset_encoded.values
X = array[:,0:108]
y = array[:,108]

# Seleção de atributos com SelectKBest
best_var = SelectKBest(score_func=f_classif, k=10)
fit = best_var.fit(X, y)    # Executa a função de pontuação em (X, y) e obtém os atributos selecionados
features = fit.transform(X) # Reduz X para os atributos selecionados

# Exibe as pontuações de cada atributos e os 90 escolhidas (com as pontuações mais altas)
np.set_printoptions(precision=3) # 3 casas decimais
#print("\nScores dos Atributos Originais:", fit.scores_)
print("\nAtributos Selecionados:", best_var.get_feature_names_out(input_features=dataset_encoded.columns[0:108]))

# Pegamos todos os atributos mesmo que de categorias diferentes que ficaram entre os 10 no SelectKBest
palavras_chave = ['age', 'education', 'capital', 'hours', 'marital', 'relationship', 'sex']

# Filtra as colunas com base nas palavras-chave
colunas_filtradas = dataset_encoded.filter(regex='|'.join(palavras_chave))

# Resultados
print('\nNúmero original de atributos:', X.shape[1])
print('\nNúmero de atributos SelectKBest:', features.shape[1])
print('\nNúmero de atributos considerando todas as categorias entre os 10 selecionados do KBest:', colunas_filtradas.shape[1])

# Separação do DataSet em base de treino e teste com 20% dos dados para teste dos modelos. Já considerado o SelectKBest onde X=features.
test_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(colunas_filtradas, y,
    test_size=test_size, shuffle=True, random_state=seed, stratify=y) # holdout com estratificação

# Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 10
kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed) # validação cruzada com estratificação


Atributos Selecionados: ['age' 'education-num' 'capital-gain' 'hours-per-week'
 'marital-status_ Married-civ-spouse' 'marital-status_ Never-married'
 'relationship_ Husband' 'relationship_ Own-child' 'sex_ Female'
 'sex_ Male']

Número original de atributos: 108

Número de atributos SelectKBest: 10

Número de atributos considerando todas as categorias entre os 10 selecionados do KBest: 37


In [11]:
# Exibe o DataFrame com as colunas filtradas
colunas_filtradas.head(10)

,age,education-num,capital-gain,capital-loss,hours-per-week,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,...,marital-status_ Widowed,occupation_ Exec-managerial,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,sex_ Female,sex_ Male
0,39,13,2174,0,40,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,50,13,0,0,13,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,38,9,0,0,40,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,53,7,0,0,40,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,28,13,0,0,40,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
5,37,14,0,0,40,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
6,49,5,0,0,16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,52,9,0,0,45,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,31,14,14084,0,50,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
9,42,13,5178,0,40,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Modelagem e Inferência
Selecionado algoritmos para problemas de classificação em aprendizado supervisionado, categórico com target em 2 classes possíveis. Salário >50k ou <=50K.
Algoritmos selecionados : Regressão Logística, KNN, Árvore de decisão, Naive Bayes, SVM e os ensambles BaggingClassifier, RandomForest, ExtraTrees, AdaBoost, GradientBoosting e Voting.
Os parâmetros da árvore de decisão num_trees e max_features foram variados e fixados em 100 e 3 que apresentou melhor resultado.

### Criação e avaliação de modelos: linha base

In [18]:
np.random.seed(7) # definindo uma semente global

# Lista que armazenará os modelos
models = []

# Criando os modelos e adicionando-os na lista de modelos
models.append(('LR', LogisticRegression(max_iter=200)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# Definindo os parâmetros do classificador base para o BaggingClassifier
base = DecisionTreeClassifier()
num_trees = 100
max_features = 3

# Criando os modelos para o VotingClassifier
bases = []
model1 = LogisticRegression(max_iter=200)
bases.append(('logistic', model1))
model2 = DecisionTreeClassifier()
bases.append(('cart', model2))
model3 = SVC()
bases.append(('svm', model3))

# Criando os ensembles e adicionando-os na lista de modelos
models.append(('Bagging', BaggingClassifier(base_estimator=base, n_estimators=num_trees)))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('Ada', AdaBoostClassifier(n_estimators=num_trees)))
models.append(('GB', GradientBoostingClassifier(n_estimators=num_trees)))
models.append(('Voting', VotingClassifier(bases)))

# Listas para armazenar os resultados
results = []
names = []

# Avaliação dos modelos
for name, model in models:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(15,10))
fig.suptitle('Comparação dos Modelos')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

LR: 0.847820 (0.004884)
KNN: 0.845017 (0.008024)
CART: 0.817413 (0.009746)
NB: 0.799946 (0.007575)
SVM: 0.803287 (0.006092)
Bagging: 0.844018 (0.007015)
RF: 0.843059 (0.004788)
ET: 0.822827 (0.004816)
Ada: 0.864021 (0.004444)


KeyboardInterrupt: ignored

### Criação e avaliação de modelos: dados padronizados e normalizados
-> Utilização de padronização e normalização no pipeline e comparação de performance entre os algoritmos de classificação escolhidos para análise da acurácia.
-> Não foi verificada a situação de underfitting
-> Os ensambles apresentaram os melhores resultados, em particular o Adaboost e o gradient_boosting.

In [ ]:
np.random.seed(7) # definindo uma semente global para este bloco

# Listas para armazenar os armazenar os pipelines e os resultados para todas as visões do dataset
pipelines = []
results = []
names = []


# Criando os elementos do pipeline

# Algoritmos que serão utilizados
reg_log = ('LR', LogisticRegression(max_iter=200))
knn = ('KNN', KNeighborsClassifier())
cart = ('CART', DecisionTreeClassifier())
naive_bayes = ('NB', GaussianNB())
svm = ('SVM', SVC())
bagging = ('Bag', BaggingClassifier(base_estimator=base, n_estimators=num_trees))
random_forest = ('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features))
extra_trees = ('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features))
adaboost = ('Ada', AdaBoostClassifier(n_estimators=num_trees))
gradient_boosting = ('GB', GradientBoostingClassifier(n_estimators=num_trees))
voting = ('Voting', VotingClassifier(bases))

# Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())


# Montando os pipelines

# Dataset original
pipelines.append(('LR-orig', Pipeline([reg_log])))
pipelines.append(('KNN-orig', Pipeline([knn])))
pipelines.append(('CART-orig', Pipeline([cart])))
pipelines.append(('NB-orig', Pipeline([naive_bayes])))
pipelines.append(('SVM-orig', Pipeline([svm])))
pipelines.append(('Bag-orig', Pipeline([bagging])))
pipelines.append(('RF-orig', Pipeline([random_forest])))
pipelines.append(('ET-orig', Pipeline([extra_trees])))
pipelines.append(('Ada-orig', Pipeline([adaboost])))
pipelines.append(('GB-orig', Pipeline([gradient_boosting])))
pipelines.append(('Vot-orig', Pipeline([voting])))

# Dataset Padronizado
pipelines.append(('LR-padr', Pipeline([standard_scaler, reg_log])))
pipelines.append(('KNN-padr', Pipeline([standard_scaler, knn])))
pipelines.append(('CART-padr', Pipeline([standard_scaler, cart])))
pipelines.append(('NB-padr', Pipeline([standard_scaler, naive_bayes])))
pipelines.append(('SVM-padr', Pipeline([standard_scaler, svm])))
pipelines.append(('Bag-padr', Pipeline([standard_scaler, bagging])))
pipelines.append(('RF-padr', Pipeline([standard_scaler, random_forest])))
pipelines.append(('ET-padr', Pipeline([standard_scaler, extra_trees])))
pipelines.append(('Ada-padr', Pipeline([standard_scaler, adaboost])))
pipelines.append(('GB-padr', Pipeline([standard_scaler, gradient_boosting])))
pipelines.append(('Vot-padr', Pipeline([standard_scaler, voting])))

# Dataset Normalizado
pipelines.append(('LR-norm', Pipeline([min_max_scaler, reg_log])))
pipelines.append(('KNN-norm', Pipeline([min_max_scaler, knn])))
pipelines.append(('CART-norm', Pipeline([min_max_scaler, cart])))
pipelines.append(('NB-norm', Pipeline([min_max_scaler, naive_bayes])))
pipelines.append(('SVM-norm', Pipeline([min_max_scaler, svm])))
pipelines.append(('Bag-norm', Pipeline([min_max_scaler, bagging])))
pipelines.append(('RF-norm', Pipeline([min_max_scaler, random_forest])))
pipelines.append(('ET-norm', Pipeline([min_max_scaler, extra_trees])))
pipelines.append(('Ada-norm', Pipeline([min_max_scaler, adaboost])))
pipelines.append(('GB-norm', Pipeline([min_max_scaler, gradient_boosting])))
pipelines.append(('Vot-norm', Pipeline([min_max_scaler, voting])))

# Executando os pipelines
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std()) # formatando para 3 casas decimais
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(25,6))
fig.suptitle('Comparação dos Modelos - Dataset orginal, padronizado e normalizado')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names, rotation=90)
plt.show()

### Otimização dos hiperparâmetros
1. Em análise dos valores de acurácia obtidos após aplicação da validação cruzada e confirmado após aplicação da normalização e padronização no pipeline,observei uma  melhor performance dos ensambles e resolvi testar a variação dos hiperparametros para o AdaBoost. O GridSearchCV apresentou como resposta Melhores hiperparâmetros: {'learning_rate': 1.0, 'n_estimators': 150}.



In [ ]:
# Tuning do AdaBoost

np.random.seed(7) # definindo uma semente global para este bloco

# Definir os hiperparâmetros que deseja otimizar
parametros = {
    'n_estimators': [50, 100, 150],  # número de estimadores
    'learning_rate': [0.1, 0.5, 1.0]  # taxa de aprendizado
}

# Criar uma instância do AdaBoostClassifier
adaboost = AdaBoostClassifier()

# Criar uma instância do GridSearchCV
grid_search = GridSearchCV(adaboost, parametros, cv=5)

# Realizar a busca em grade (grid search) com validação cruzada
grid_search.fit(X_train, y_train)

# Visualizar os melhores hiperparâmetros encontrados
melhores_hiperparametros = grid_search.best_params_
print("Melhores hiperparâmetros:", melhores_hiperparametros)

## Finalização do Modelo e Avaliação de Resultados
-> Escolhi a métrica de acurácia, por ser adequada para o problema categórico em questão com 2 classes. Como é supervisionado e possuímos a base de treino e teste com o target real e os valores preditos , essa métrica é adequada para verificação da qualidade e performance do modelo na predição das classes, quanto maior a taxa de acerto melhor.
-> Nesta seção será treinado o modelo com a base de treino de forma segregada e posteriormente testado com a base de teste. Mantendo o isolamento entre os dados de treino e teste.

In [ ]:
# Avaliação do modelo com o conjunto de testes

# Preparação do modelo
adaboost = AdaBoostClassifier(learning_rate=1.0,n_estimators=150)
adaboost.fit(X_train, y_train)

name = 'adaboost'

cv_results = cross_val_score(adaboost, X_train, y_train, cv=kfold, scoring=scoring)
msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(msg)


# Estimativa da acurácia no conjunto de teste
predictions = adaboost.predict(X_test)
print(accuracy_score(y_test, predictions))